<a href="https://colab.research.google.com/github/Sachinsachin21/Deep_learning/blob/main/LSTM_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [ ]:
s = pd.read_csv("IMDB Dataset.csv")
s.head(10)

In [ ]:
s["sentiment"].replace({"positive":1,"negative":0},inplace=True)
s.head()

In [ ]:
x = np.array(s["review"].values)
y = np.array(s["sentiment"].values)

In [ ]:
x_filtered = []

for review in x:
    review = review.lower()
    for i in review:
        punc = '''  !()-[]{};:'"\,<>./?@#$%^&*_~  '''
        if i in punc :
            review = review.replace(i, " ")
    x_filtered.append(review)
x_filtered

In [ ]:
vocalbulary_size = 5000
onehot_encoded = [one_hot(review,vocalbulary_size) for review in x_filtered]
onehot_encoded

In [ ]:
max_length=500
x_padded = pad_sequences(onehot_encoded,max_length,padding="post")
x_padded

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_padded,y,test_size=0.2)

In [ ]:
model = Sequential()
embeded_vector_size = 35
model.add(Embedding(vocalbulary_size,embeded_vector_size,input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"])

In [ ]:
model.build((None, max_length))
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=30, validation_data=(x_test, y_test))

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
model.save('sentiment_model.h5')

In [ ]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(x_filtered)
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
def get_sentiment(sentence: str):
    if not isinstance(sentence, str):
        raise ValueError("Input needs to be of type 'str'")
    sentence = sentence.lower()
    sentence = ''.join(char for char in sentence if char.isalnum() or char.isspace())
    with open('tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=500)
    model = load_model('sentiment_model.h5')
    predicted = model.predict(padded_sequence)
    sentiment = "Positive" if predicted[0][0] > 0.5 else "Negative"
    return sentiment

print(get_sentiment("The software crashed frequently, causing a lot of frustration and lost work."))